In [1]:
import pandas
import simplemma
import spacy
import nltk
from collections import Counter
import emoji

In [2]:
model_es = spacy.load("es_core_news_sm")


In [3]:
df_verbs = pandas.read_csv("./resources/verbs_database.csv")
df_verbs.head(3)


,infinitive,infinitive_english,mood,mood_english,tense,tense_english,verb_english,form_1s,form_2s,form_3s,form_1p,form_2p,form_3p,gerund,gerund_english,pastparticiple,pastparticiple_english
0,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Presente,Present,"I abandon, am abandoning",abandono,abandonas,abandona,abandonamos,abandonáis,abandonan,abandonando,abandoning,abandonado,abandoned
1,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Futuro,Future,I will abandon,abandonaré,abandonarás,abandonará,abandonaremos,abandonaréis,abandonarán,abandonando,abandoning,abandonado,abandoned
2,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Imperfecto,Imperfect,"I was abandoning, used to abandon, abandoned",abandonaba,abandonabas,abandonaba,abandonábamos,abandonabais,abandonaban,abandonando,abandoning,abandonado,abandoned


In [4]:
def diversidad_lexica(texto):
    tokens = simplemma.simple_tokenizer(texto)
    return len(set(tokens)) / len(tokens)


def numero_oraciones(text: str) -> int:
    oraciones = nltk.tokenize.sent_tokenize(text, language="spanish")
    return len(oraciones)


def palabras_promedio_oraciones(text: str) -> float:
    oraciones = nltk.tokenize.sent_tokenize(text, language="spanish")
    longitud_promedio = sum(len(oracion.split()) for oracion in oraciones) / len(oraciones)
    return longitud_promedio


def numero_oraciones(text: str) -> int:
    oraciones = nltk.tokenize.sent_tokenize(text, language="spanish")
    return len(oraciones)


def count_gramatical_categories_and_forms(text: str) -> pandas.Series:
    GRAMATICAL_CATEGORIES_AND_FORMS = {
        "stop_words": 0,
        "alpha_words": 0,
        "NOUN": 0,
        "ADJ": 0,
        "VERB": 0,
        "PRON": 0,
        "ADV": 0,
        "DET": 0,
        "CCONJ": 0,
        "INTJ": 0,
        "SCONJ": 0,
        "AUX": 0,
        "PUNCT": 0,
        "PROPN": 0,
        "NUM": 0,
        "form_1s": 0,
        "form_2s": 0,
        "form_3s": 0,
        "form_1p": 0,
        "form_2p": 0,
        "form_3p": 0,
        "gerund": 0,
        "infinitive": 0,
        "Presente": 0,
        "Futuro": 0,
        "Imperfecto": 0,
        "Pretérito": 0,
        "Condicional": 0,
        "Presente perfecto": 0,
        "Futuro perfecto": 0,
        "Pluscuamperfecto": 0,
        "Pretérito anterior": 0,
        "Condicional perfecto": 0,
        "Indicativo": 0,
        "Subjuntivo": 0,
        "Imperativo Afirmativo": 0,
        "Imperativo Negativo": 0,
    }
    doc = model_es(text.lower())
    for token in doc:
        if token.is_stop:
            GRAMATICAL_CATEGORIES_AND_FORMS["stop_words"] += 1
        if token.is_alpha:
            GRAMATICAL_CATEGORIES_AND_FORMS["alpha_words"] += 1
        if token.pos_ in GRAMATICAL_CATEGORIES_AND_FORMS:
            GRAMATICAL_CATEGORIES_AND_FORMS[token.pos_] += 1
        if token.pos_ == "VERB":
            try:
                idx, form = df_verbs.where(df_verbs == token.text.lower()).stack().index[0]
                if form in GRAMATICAL_CATEGORIES_AND_FORMS:
                    GRAMATICAL_CATEGORIES_AND_FORMS[form] += 1
                GRAMATICAL_CATEGORIES_AND_FORMS[df_verbs.loc[idx, "tense"]] += 1
                GRAMATICAL_CATEGORIES_AND_FORMS[df_verbs.loc[idx, "mood"]] += 1
            except:
                pass
    return pandas.Series(GRAMATICAL_CATEGORIES_AND_FORMS)


In [5]:
def label_num_clasificacion(label: str) -> int:
    LABELS = {"LLM": 0, "human": 1}
    return LABELS[label]


def label_num_fuente(label: str) -> int:
    LABELS = {"Cuentos": 0, "Discursos": 1, "Ensayos": 2, "Fabulas": 3, "Noticias": 4, "Poemas": 5, "Tweets": 6}
    return LABELS[label]


In [6]:
df = pandas.read_excel("./database_v2.xlsx")
df["fuente_num"] = df["Tipo de fuente"].apply(label_num_fuente)
df["clasificacion_num"] = df["Clasificacion"].apply(label_num_clasificacion)
df


,Texto,Tipo de fuente,Clasificacion,fuente_num,clasificacion_num
0,AL PIE DEL RÍO Había una vez una madre y su hi...,Cuentos,LLM,0,0
1,Bulto de salHabía una vez un hombre llamado Bu...,Cuentos,LLM,0,0
2,De por qué el armadillo lleva a cuestas una pe...,Cuentos,LLM,0,0
3,"Domingo 7Había una vez dos amigos jorobados, u...",Cuentos,LLM,0,0
4,El burrito y la TunaHabía una vez un hombre ll...,Cuentos,LLM,0,0
...,...,...,...,...,...
1839,RT @RevistaSemana: #SemanaNoticias | La situac...,Tweets,human,6,1
1840,@caritosarita No es cierto. Por favor.,Tweets,human,6,1
1841,RT @VickyDavilaH: La gran encuesta de Cifras y...,Tweets,human,6,1
1842,RT @VickyDavilaH: No tuvimos suerte con la señ...,Tweets,human,6,1


In [8]:
df["Tipo de fuente"].unique()

array(['Cuentos', 'Discursos', 'Ensayos', 'Fabulas', 'Noticias', 'Poemas',
       'Tweets'], dtype=object)

In [14]:
df["diversidad_lexica"] = df["Texto"].apply(diversidad_lexica)
df["numero_oraciones"] = df["Texto"].apply(numero_oraciones)
df["palabras_promedio_oraciones"] = df["Texto"].apply(palabras_promedio_oraciones)
df["numero_oraciones"] = df["Texto"].apply(numero_oraciones)
df[
    [
        "stop_words",
        "alpha_words",
        "noun",
        "adj",
        "verb",
        "pron",
        "adv",
        "det",
        "cconj",
        "intj",
        "sconj",
        "aux",
        "punct",
        "propn",
        "num",
        "form_1s",
        "form_2s",
        "form_3s",
        "form_1p",
        "form_2p",
        "form_3p",
        "gerund",
        "infinitive",
        "Presente",
        "Futuro",
        "Imperfecto",
        "Pretérito",
        "Condicional",
        "Presente perfecto",
        "Futuro perfecto",
        "Pluscuamperfecto",
        "Pretérito anterior",
        "Condicional perfecto",
        "Indicativo",
        "Subjuntivo",
        "Imperativo Afirmativo",
        "Imperativo Negativo",
    ]
] = (
    df["Texto"].apply(count_gramatical_categories_and_forms).reset_index(drop=True)
)
df["emoji_count"] = df["Texto"].apply(lambda text: emoji.emoji_count(text))
df["hashtags_count"] = df["Texto"].apply(lambda text: text.count("#"))
df["links_count"] = df["Texto"].apply(lambda text: text.count("http"))
df["tags_count"] = df["Texto"].apply(lambda text: text.count("@"))
df["retweets_count"] = df["Texto"].apply(lambda text: text.count("RT"))
df


,Texto,Tipo de fuente,Clasificacion,fuente_num,clasificacion_num,diversidad_lexica,numero_oraciones,palabras_promedio_oraciones,stop_words,alpha_words,...,Condicional perfecto,Indicativo,Subjuntivo,Imperativo Afirmativo,Imperativo Negativo,emoji_count,hashtags_count,links_count,tags_count,retweets_count
0,AL PIE DEL RÍO Había una vez una madre y su hi...,Cuentos,LLM,0,0,0.424000,17,26.117647,252,436,...,0,48,2,0,0,0,0,0,0,0
1,Bulto de salHabía una vez un hombre llamado Bu...,Cuentos,LLM,0,0,0.382479,18,23.222222,242,412,...,0,55,7,0,0,0,0,0,0,0
2,De por qué el armadillo lleva a cuestas una pe...,Cuentos,LLM,0,0,0.423729,12,31.000000,229,366,...,0,42,1,0,0,0,0,0,0,0
3,"Domingo 7Había una vez dos amigos jorobados, u...",Cuentos,LLM,0,0,0.387914,24,18.583333,250,435,...,0,49,2,0,0,0,0,0,0,0
4,El burrito y la TunaHabía una vez un hombre ll...,Cuentos,LLM,0,0,0.416495,19,21.947368,230,406,...,0,59,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,RT @RevistaSemana: #SemanaNoticias | La situac...,Tweets,human,6,1,0.956522,1,22.000000,12,20,...,0,0,0,0,0,0,1,0,1,1
1840,@caritosarita No es cierto. Por favor.,Tweets,human,6,1,0.875000,2,3.000000,4,5,...,0,0,0,0,0,0,0,0,1,0
1841,RT @VickyDavilaH: La gran encuesta de Cifras y...,Tweets,human,6,1,0.904762,1,20.000000,10,18,...,0,1,0,0,0,0,0,1,1,1
1842,RT @VickyDavilaH: No tuvimos suerte con la señ...,Tweets,human,6,1,0.960000,2,11.000000,9,21,...,0,1,0,0,0,0,1,0,1,1


In [15]:
df.to_csv("./database_no_lexico.csv", index=False)